In [1]:
import pandas as pd
path=r'C:\\Users\\student\\Desktop\\공부\\멀캠TIL\\dataset\\python\\the-movies-dataset'

data=pd.read_csv(path+'\\movies_metadata.csv')

C:\Users\student\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
data=data.head(20000)

In [4]:
# 결측값 처리
data['overview'].isnull().sum()

135

In [5]:
data['overview'] = data['overview'].fillna('')

In [6]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(data['overview'])
# overview에 대해서 tf-idf 수행
tfidf_matrix.shape
# (20000 영화, 47487 단어)

(20000, 47487)

In [7]:
from sklearn.metrics.pairwise import linear_kernel
cos_sim=linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
title_index=pd.Series(data.index, index=data['title']).drop_duplicates()
title_index.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [13]:
def movie_recommendations(title, cos_sim=cos_sim):
    # 해당 영화의 index 추출
    idx=title_index[title]

    # 모든 영화에 대해서 해당 영화와의 유사도 추출
    sim_scores=list(enumerate(cos_sim[idx]))

    # 유사도에 따라 영화 정렬
    sim_scores=sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화/index 추출
    sim_scores=sim_scores[1:11]
    movie_index=[i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화 제목 출력
    return data['title'].iloc[movie_index]

movie_recommendations('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object